# Resources
https://www.wnycstudios.org/podcasts/radiolab/segments/104010-one-good-deed-deserves-another

https://fivethirtyeight.com/features/how-to-win-a-trade-war/

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from DefaultBots import *     # {Player, TitForTat, Satan, Jesus, Arbitrary, MassiveRetalitoryStrike, Tester}
## import more bots here

In [ ]:
def run_one_game(player1, player2, player1_prev_choices, player2_prev_choices):
    
    p1_ans = player1.evaluate_round(player2_prev_choices)
    p2_ans = player2.evaluate_round(player1_prev_choices)
    
    return p1_ans, p2_ans
    
def run_many_games(player1, player2, n_games=200, verbose=False):
    outcome_array = np.zeros(n_games)
    p1_choices    = np.array([])
    p2_choices    = np.array([])
    
    for i in np.arange(n_games):
        p1_choice, p2_choice = run_one_game(player1, player2, p1_choices, p2_choices)
        p1_choices = np.append(p1_choices, p1_choice)
        p2_choices = np.append(p2_choices, p2_choice)
        
        p1_defect = (p1_choice == 1)
        p2_defect = (p2_choice == 1)
        
        outcome = -1
        if (p1_defect and p2_defect):
            outcome =  0
        if (p1_defect and not p2_defect):
            outcome =  1
        if (not p1_defect and p2_defect):
            outcome =  2
        if (not p1_defect and not p2_defect):
            outcome =  3
        outcome_array[i] = outcome
    
    if (verbose):
        print("Player",player1.name,"chose:",p1_choices)
        print("Player",player2.name,"chose:",p2_choices)
    return outcome_array

def determine_winner(player1, player2, outcome_array, verbose=True):
    p1_wins = np.sum(outcome_array == 1)
    p2_wins = np.sum(outcome_array == 2)
    both_lose = np.sum(outcome_array == 0) # both defect
    both_win  = np.sum(outcome_array == 3) # both remain
    
    pts_both_defect   = 0.0
    pts_both_remain   = 1.0
    pts_opposite_win  = 2.0
    pts_opposite_lose = 0.0
    
    p1_score = p1_wins*pts_opposite_win + both_win*pts_both_remain + both_lose*pts_both_defect + p2_wins*pts_opposite_lose
    p2_score = p2_wins*pts_opposite_win + both_win*pts_both_remain + both_lose*pts_both_defect + p1_wins*pts_opposite_lose
    
    if   (p1_score > p2_score):
        if (verbose):
            print("The winner is:",player1.name,", with a score of",p1_score,"out of 400 total (",100*p1_score/400.,"%)")
            print("Loser (",player2.name,") scored:",p2_score)
            print(player1.name,"beat",player2.name,"by",p1_score-p2_score)
        return player1, p1_score, p2_score
    elif (p2_score > p1_score):
        if (verbose):
            print("The winner is:",player2.name,", with a score of",p2_score,"out of 400 total (",100*p2_score/400.,"%)")
            print("Loser (",player1.name,") scored:",p1_score)
            print(player1.name,"lost to",player2.name,"by",p1_score-p2_score)
        return player2, p1_score, p2_score
    else:
        if (verbose):
            print("Tie, both received",p1_score,"points")
        return Player(), p1_score, p2_score

In [ ]:
# Example of head-to-head competition
_p1 = JESUS(1)
_p2 = SATAN(2)
results = run_many_games(_p1,_p2)
winner, p1_score, p2_score = determine_winner(_p1,_p2,results)
n, bins, patches = plt.hist(results)

In [ ]:
N_bots       = 6

def MakeBotArray(idx):
    temp_array = np.zeros(N_bots, dtype=object)
    # Default Bots - 6
    temp_array[ 0] = TitForTat(idx)
    temp_array[ 1] = Satan(idx)
    temp_array[ 2] = Jesus(idx)
    temp_array[ 3] = Arbitrary(idx)
    temp_array[ 4] = MassiveRetalitoryStrike(idx)
    temp_array[ 5] = Tester(idx)
    # Add more bots here (be sure to change N_bots to match)
    return temp_array

BotArray1 = MakeBotArray(1)
BotArray2 = MakeBotArray(2)

In [ ]:
def RunTheGauntlet(p1, VERBOSE=False):
    p2_array       = BotArray2
    score_array_1  = np.zeros(N_bots)
    score_array_2  = np.zeros(N_bots)
    oppname_arr    = np.zeros(N_bots, dtype=object)

    for i in np.arange(N_bots):
        results = run_many_games(p1,p2_array[i])
        winner, p1_score, p2_score = determine_winner(p1,p2_array[i],results,verbose=False)
        score_array_1[i] = p1_score
        score_array_2[i] = p2_score
        oppname_arr[i]   = p2_array[i].name
        if VERBOSE:
            if   (winner.name == p1.name):
                print(p1.name, "wins vs" , p2_array[i].name, "scoring", p1_score, "points (", p2_score, ")" )
            elif (winner.name == p2_array[i].name):
                print(p1.name, "loses vs", p2_array[i].name, "scoring", p1_score, "points (", p2_score, ")" )
            else:
                print(p1.name, "ties wth", p2_array[i].name, "scoring", p1_score, "points (", p2_score, ")" )
    return p1.name, oppname_arr, score_array_1, score_array_2

In [ ]:
_p1 = TitForTat()
name, opp_names, scores, opp_scores = RunTheGauntlet(_p1)

scores = np.append(scores, np.sum(scores))
opp_names = np.append(opp_names, "TOTAL")
score_data = np.array([scores])

In [ ]:
# Generate a pandas dataframe using the output for player1's guantlet
# here, we're indexing by the name of player1, and each column is player2's name
# the entry in each cell is the score for player1
df = pd.DataFrame(score_data, columns=opp_names, index=[name])
df.loc[name]

In [ ]:
_p1 = BotArray1[0]

# Run the first trial of the gauntlet
name, opp_names, scores, opp_scores = RunTheGauntlet(_p1)

# Create a dictionary for the result of each mathcup
p1_dict = {'player1': name * np.ones(len(scores), dtype=object), 
           'player2': opp_names, 
           'scores1': scores, 
           'scores2': opp_scores}

# Turn the dictionary for this matchup into a dataframe
# Here, we're indexing each matchup, with columns player1 name, player2 name, player1 score, player2 score
# There should be N_bots**2 entries when we're done filling this (after loop)
df = pd.DataFrame(p1_dict)

# Loop over all the other players to be player1 in a gauntlet of matchups
for i in np.arange(N_bots-1):
    # Get the right bot
    _p1 = BotArray1[i+1]
    
    # Run the game
    name, opp_names, scores, opp_scores = RunTheGauntlet(_p1)

    # Create an identical dictionary as above
    p1_dict = {'player1': name * np.ones(len(scores), dtype=object), 
               'player2': opp_names, 
               'scores1': scores, 
               'scores2': opp_scores}
    
    # Turn our new dictionary into a dataframe
    df2 = pd.DataFrame(p1_dict)
    
    # Append the new dataframe to the one we created outside the loop
    df  = df.append(df2)

In [ ]:
df # show me the data frame in a pretty table

In [ ]:
# Verify there are as many entries as we expect
# Each bot went head to head with every other bot and itself
# So we expect there to be N_bots*N_bots rows in our dataframe
print(len(df))
print(N_bots*N_bots)

In [ ]:
# Let's look at a specific player
p1_name_now = "arbitrary-1"
# p1_name_now = "biased-statistical-striker-1"

In [ ]:
# Now get the scores for a given player1 in the matchup
p1_scores = df.loc[ df['player1'] == p1_name_now , "scores1"].to_numpy()
print("Total:",np.sum(p1_scores))

In [ ]:
# Which players did player1 win against
win_series = df.loc[ (df['player1'] == p1_name_now) & (df['scores1'] > df['scores2']), ["player2","scores1","scores2"]]
N_wins = len(win_series.to_numpy())
print(win_series)
print("Total wins:",N_wins)

In [ ]:
# Which players did player1 lose against
loss_series = df.loc[ (df['player1'] == p1_name_now) & (df['scores1'] < df['scores2']), ["player2","scores1","scores2"]]
N_losses = len(loss_series.to_numpy())
print(loss_series)
print("Total losses:",N_losses)

In [ ]:
# Which players did player1 tie with
tie_series = df.loc[ (df['player1'] == p1_name_now) & (df['scores1'] == df['scores2']), ["player2","scores1","scores2"]]
N_ties = len(tie_series.to_numpy())
print(tie_series)
print("Total ties:",N_ties)

In [ ]:
# Did we count all the games
N_games = N_wins+N_losses+N_ties
print("Total competitors:", N_games)
print("That","is" if (N_games==N_bots) else "is not","the right number")

In [ ]:
# Let's look for the top 10 bots
total_scores = np.zeros(N_bots)
for i in np.arange(N_bots):
    total_scores[i] = np.sum(df.loc[ df['player1'] == BotArray1[i].name , "scores1"].to_numpy())
arg_ans = np.argsort(total_scores)
sorted_scores = total_scores[arg_ans]

for i in np.arange(10):
    idx = N_bots - 10 + i
    _name = BotArray1[arg_ans[idx]].name
    _score = sorted_scores[idx]
    print("Place\t",N_bots-idx,"\t",_name,"\t",_score,"\t")

In [ ]:
arr1 = np.array([5,2,6,89,3,56,8,3,12,6])
print( np.sort(arr1)  )

arg1 = np.argsort(arr1)
print( arg1 )

print( arr1[arg1] )
